<a href="https://colab.research.google.com/github/SedoyChloric/work_in_collab/blob/test/SVR_QSPR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import numpy as np
from google.colab import auth
auth.authenticate_user()
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [6]:
alphabet = "abcdefghijklmnopqrstuvwxyz"
clusterisation_column = 8
idk = "Я не знаю таких цифр"

Обработка данных:

In [7]:
def prepare_clusters(winner_coordinates, learning_rate, sigma, iterations, column, frequency_threshold):
  col = calculate_column(column)
  title = np.array([[np.round(learning_rate, 4), np.round(sigma, 4), iterations, f'=COUNTUNIQUE({col}6:{col})', f'="№"&столбец({col}1)-7']])
  unique_coords, cluster_labels, labels_frequency = np.unique(winner_coordinates, axis=0, return_inverse=True, return_counts=True)
  for i, freq in enumerate(labels_frequency):
    if freq < frequency_threshold:
      cluster_labels[cluster_labels == i] = -1
  return np.append(title, [cluster_labels], axis=1)

def to_list(array):
  return array.reshape(-1, 1).tolist()

def to_normalize_except_material(array):
  min_val = np.min(array[:, 1:])
  max_val = np.max(array[:, 1:])
  normed_array = (array[:, 1:] - min_val) / (max_val - min_val)
  return np.column_stack((array[:, 0], normed_array))

def to_normalize_by_row(array):
  normed_array = np.zeros(array.shape)
  for i in range(len(array)):
    min_val = np.min(array[i])
    max_val = np.max(array[i])
    normed_array[i] = array[i]
    normed_array[i] = (array[i] - min_val) / (max_val - min_val)
  return normed_array

def trim_table(worksheet, threshold_value, start, end):
  unsupervised_data = np.array(worksheet.get_all_values())
  data = unsupervised_data[start:end][:].astype('float64')
  columns_to_delete = np.zeros(len(data[0]), dtype=bool)
  columns_to_delete[data[0] >= threshold_value] = True
  columns_to_delete[data[1] >= threshold_value] = True
  unsupervised_data = np.delete(unsupervised_data, columns_to_delete, axis=1)
  worksheet.clear()
  worksheet.update(unsupervised_data.tolist(), f'a1')

Работа с гугл-таблицами

In [8]:
def calculate_column(number, alphabet=alphabet):
  column = ''
  while number > 0:
    number, remainder = divmod(number - 1, 26)
    column = alphabet[remainder] + column
  return column

def ccell(column, row): #calculatecell
  return ''.join([calculate_column(column), str(row)])

def get_data_from_googlesheet(google_spreedsheet, sheet_name):
  table = np.array(google_spreedsheet.worksheet(sheet_name).get_all_values()) #Лист преобразуется в массив
  name_of_properties = table[0][3:-1] # Получаем наименования свойств (со второго до предпоследнего)
  table = np.transpose(table) #Транспонируем
  id = table[1][1:] #Получаем лист наименований
  source = table[-1][1:] #Получаем лист сурсов
  properties = table[3:-1][0:] #Получаем лист свойств, который нужно будет снова транспонировать
  return id, source, np.transpose(np.delete(properties, 0, 1).astype('float64')), name_of_properties

def prepare_googlesheet_for_nd(id_array, source_array, properties_values, name_of_properties, worksheet, row_bias):
  worksheet.update(to_list(np.arange(len(id_array)+1)), f'a{row_bias}:a')
  worksheet.update(to_list(id_array), f'b{row_bias+1}:b')
  worksheet.update([name_of_properties.tolist()], f'c{row_bias}')
  worksheet.update(properties_values.tolist(), f'c{row_bias+1}')
  worksheet.update_acell(f'b{row_bias}]', "Наименование точки")
  worksheet.update_acell(f'{calculate_column(len(name_of_properties)+3)}{row_bias}', "Кластеризация")
  return

def prepare_googlesheet_for_1d(id_array, source_array, property_values, name_of_property, worksheet):
  worksheet.update(to_list(np.arange(len(id_array)+1)), 'a1:a')
  worksheet.update(to_list(id_array), 'b2:b')
  worksheet.update(to_list(property_values), 'c2:c')
  worksheet.update_acell('b1', name_of_property)
  worksheet.update_acell('c1', "Значение")
  worksheet.update_acell('d1', "Кластеризация")
  return

def prepare_googlesheet_for_starting(worksheet):
  array = [["Kernel: linear"], ["C"], ["epsilon"], ["gamma"], ["MSE"]]
  worksheet.update([["Kernel: linear"], ["C"], ["epsilon"], ["gamma"], ["MSE"]], 'a2')
  return

def import_result_in_googlesheet_neuro(packed_clusters, worksheet, row_bias):
  clusterisation = worksheet.find('Кластеризация').col
  column = calculate_column(clusterisation)
  worksheet.update_acell(f'{calculate_column(clusterisation+len(packed_clusters[0]))}{row_bias}', 'Кластеризация')
  worksheet.update(packed_clusters.tolist(), f'{column}{row_bias}', value_input_option='USER_ENTERED')
  return

def import_result_in_googlesheet_dbscan(packed_clusters, worksheet, radius, minimum_points):
  column = worksheet.find('Кластеризация').col
  worksheet.update_acell(f'{calculate_column(column)}1', f'R={np.round(radius, 4)}; mp={np.round(minimum_points, 4)}')
  worksheet.update_acell(f'{calculate_column(column+1)}1', 'Кластеризация')
  worksheet.update(to_list(packed_clusters), f'{calculate_column(column)}2:{calculate_column(column)}')
  return

In [9]:
print(f'{ccell(2,1)}')

b1


In [34]:
data_spreadsheet = gc.open_by_key('199JJj9XfyCkrX9I6I0PKK07vmkfNWT2OPLLX1OSwT8s')
experiment_spreadsheet = gc.open_by_key('199JJj9XfyCkrX9I6I0PKK07vmkfNWT2OPLLX1OSwT8s')
indices_id_array, source_array, indices, name_of_indices = get_data_from_googlesheet(data_spreadsheet, "Статья1QSPR")
properties_id_array, source_array, properties, name_of_properties = get_data_from_googlesheet(data_spreadsheet, "Статья1св-ва")

In [16]:
try:
  sheet_name = input("Результаты будут импортированы в гугл-таблицу. \nИмя листа будет: ")
  worksheet = experiment_spreadsheet.add_worksheet(title=sheet_name, rows=1000, cols=1000)
  worksheet.update([name_of_indices.tolist()], ccell(2, 1))
except gspread.exceptions.APIError:
  worksheet = experiment_spreadsheet.worksheet(sheet_name)


Результаты будут импортированы в гугл-таблицу. 
Имя листа будет: Нормированные QSPR и Св-ва 2


Основной блок реализации

In [36]:
indices_scaler = StandardScaler()
properties_scaler = StandardScaler()
indices_train, indices_test, properties_train, properties_test = train_test_split(indices, properties, test_size=0.2, shuffle=False)
indices_train_scaled = indices_scaler.fit_transform(indices_train)
indices_test_scaled =  indices_scaler.transform(indices_test)
properties_train_scaled = properties_scaler.fit_transform(properties_train)
properties_test_scaled =  properties_scaler.transform(properties_test)
indices_train_scaled = indices_train_scaled.T
indices_test_scaled = indices_test_scaled.T
properties_train_scaled = properties_train_scaled.T
properties_test_scaled = properties_test_scaled.T

In [37]:
print(indices_train_scaled.shape)

(8, 12)


In [50]:
svr = SVR()
param_grid = {
    'kernel': ['linear'],
    'C':[0.01, 0.1, 1, 10, 100, 1000],
    'gamma': ['scale', 'auto', 0.001, 0.01, 0.1, 1],
    'epsilon': [0.01, 0.1, 0.2, 0.5]
    }

best_params = []
best_scores = []
trained_svrs = []

In [ ]:
subrow = 2
subcol = 10
row = 2
column = 2
for i in range(properties_train.shape[0]):
  print(f"Training SVR for property {name_of_properties[i]}...")
  title = np.array([[f"{name_of_properties[i]}"], ["C"], ["epsilon"], ["gamma"], ["MSE"], ["a0"], ["a1"]])
  worksheet.update(title.tolist(), f'{calculate_column(column-1)}{row}')
  worksheet.update_acell(f'{calculate_column(subcol)}{subrow}', f'{name_of_properties[i]}')
  iteration = np.empty_like(title)
  for j in range(indices_train.shape[1]):
    print(f"By molecular indice {name_of_indices[j]}...")
    grid_search = GridSearchCV(svr, param_grid, cv=5, scoring='neg_mean_squared_error', verbose=0, n_jobs=-1)
    grid_search.fit(indices_train[:, j].reshape(-1, 1), properties_train[:, i])
    subiteration = np.array([["X"]])
    subiteration = np.append(subiteration, np.array(list(grid_search.best_params_.values())[0:3]).reshape(-1, 1), axis=0)
    subiteration = np.append(subiteration, [[np.round(grid_search.best_score_, 4)]], axis=0)
    subiteration = np.append(subiteration, np.round(np.array([grid_search.best_estimator_.intercept_]), 6), axis=0)
    subiteration = np.append(subiteration, np.round(grid_search.best_estimator_.coef_, 6), axis=0)
    iteration = np.hstack((iteration, subiteration))
  iteration = np.delete(iteration, 0, axis=1)
  worksheet.update_acell(f'{calculate_column(subcol+1)}{subrow}', f'=ИНДЕКС(b1:i1, Поискпоз(макс({calculate_column(column)}{row+4}:{calculate_column(column+iteration.shape[1]-1)}{row+4}), {calculate_column(column)}{row+4}:{calculate_column(column+iteration.shape[1]-1)}{row+4}, 0))')
  subrow += 1
  worksheet.update(iteration.tolist(), f'{calculate_column(column)}{row}', value_input_option='USER_ENTERED')
  row += title.shape[0]


In [ ]:
array = np.array((0, 0, 0 ))
print(to_list(array))

[[0], [0], [0]]


In [ ]:
# Inverse transform the scaled properties_test and properties_predicted
properties_test_original = scaler.inverse_transform(np.hstack((indices_test, properties_test)))[:, indices.shape[1]:]
properties_predicted_original = scaler.inverse_transform(np.hstack((indices_test, properties_predicted.reshape(-1, properties_test.shape[1]))))[:, indices.shape[1]:]


mse = mean_squared_error(properties_test_original, properties_predicted_original)
r2 = r2_score(properties_test_original, properties_predicted_original)
print(f"MSE: {mse:.4f}")
print(f"R^2: {r2:.4f}")

MSE: 20036.3098
R^2: -2.3588


In [ ]:
normed_data = to_normalize_except_material(data)
column = 8
for i in range(2, 6):
  learning_rate = i*0.1
  for j in range(1, 9):
    sigma = j*0.2
    packed_clusters = np.zeros((1, len(normed_data)+5))
    for k in range(4, 13):
      iterations = k*50
      som = MiniSom(10, 10, len(normed_data[0]), sigma=sigma, learning_rate=learning_rate)
      som.train_batch(normed_data, iterations)
      winner_coordinates = np.array([som.winner(x) for x in normed_data])
      clusterisation_results = prepare_clusters(winner_coordinates, learning_rate, sigma, iterations, column, 3)
      packed_clusters = np.append(packed_clusters, clusterisation_results, axis=0)
      column +=1
    packed_clusters = np.delete(packed_clusters, 0, axis=0)
    import_result_in_googlesheet_neuro(packed_clusters.T, worksheet, 1)


In [ ]:
column = 8
for i in range(1, 4):
  learning_rate = i*0.1
  for j in range(1, 4):
    sigma = j*0.2
    packed_clusters = np.zeros((1, len(normed_data)+5))
    for k in range(2, 4):
      iterations = k*100
      som = MiniSom(10, 10, len(normed_data[0]), sigma=sigma, learning_rate=learning_rate)
      som.train_batch(train_data, iterations)
      winner_coordinates = np.array([som.winner(x) for x in train_data_spec])
      winner_coordinates = np.append(winner_coordinates, np.array([som.winner(x) for x in test_data]), axis=0)
      clusterisation_results = prepare_clusters(winner_coordinates, learning_rate, sigma, iterations, column, 0)
      packed_clusters = np.append(packed_clusters, clusterisation_results, axis=0)
      column +=1
    packed_clusters = np.delete(packed_clusters, 0, axis=0)
    import_result_in_googlesheet_neuro(packed_clusters.T, worksheet, 1)


ValueError: operands could not be broadcast together with shapes (4,) (10,10,5) 

In [ ]:
print("Есть на просторах интернета одна гугл-таблица, в которой собрано немного про характеристики аэрогелей.\n"
  "В этой прекрасной гугл-таблице насчитываются характеристики уже", len(id_array), "аэрогелей\n"
  "Эта программа может разбивать аэрогели на группы при помощи магии, которая зовётся MiniSOM\n"
  "Магия работает по таким характеристикам, как", name_of_properties)
while True:
  print("1. Кронк, жми на рычаг просмотра данных!"
  "\n2. Кронк, жми на рычаг кластеризации!"
  "\n3. Кронк, жми на рычаг в лабораторию!")
  try:
    choice = int(input("Нажмите на рычаг: "))
    if choice == 1:
      for i in range(len(id_array)-1):
        print("Точка", id_array[i], "с координатами:\n", data[i])
    elif choice == 2:
      try:
        sheet_name = input("Результаты будут импортированы в гугл-таблицу. \nИмя листа будет: ")
        worksheet = experiment_spreadsheet.add_worksheet(title=sheet_name, rows=len(id_array)+5, cols=2000)
        prepare_googlesheet_for_nd(id_array, source_array, data, name_of_properties, worksheet)
      except gspread.exceptions.APIError:
        worksheet = experiment_spreadsheet.worksheet(sheet_name)
      while True:
        print("Задать сигму\n"
              "Одного единого универсального значения нет!\n")
        while True:
            try:
              sigma = float(input("Введите сигму: "))
              break
            except ValueError:
              print(idk)
        print("Задать скорость обучения\n",
              "1. Использовать 3\n",
              "2. Задать число самому\n")
        choice_2 = input("Введите опцию: ")
        while True:
          if choice_2 == '1':
            learning_rate = 0.5
            break
          elif choice_2 == '2':
            while True:
              try:
                learning_rate = int(input("Введите минимальное количество точек (Рекомендуется брать от 5 до 2): "))
                break
              except ValueError:
                print(idk)
          else:
            print(idk)
        #normed_data = to_normalize_except_material(data)
        normed_data = to_normalize_except_material(data)
        som = MiniSom(1, len(normed_data[:]), len(normed_data[0, :]), sigma=sigma, learning_rate=learning_rate)
        som.train_batch(normed_data, 100)
        winner_coordinates = np.array([som.winner(x) for x in data]).T
        packed_clusters = pack_clusters(data, winner_coordinates[1], 3)
        import_result_in_googlesheet_neuro(packed_clusters, worksheet, sigma, learning_rate)
        print(f"Данные в гугл-таблице на листе '{sheet_name}' обновлены!\n")
        while True:
          print("Попробовать изменить радиус и/или минамальное количество точек и попробовать снова?\n"
          "1. Да\n"
          "2. Нет\n")
          choice_2 = input("Введите опцию: ")
          if choice_2 == '1':
            break
          elif choice_2 == '2':
            break
          else:
            print(idk)
        if choice_2 == '2':
            break
    elif choice == 3:
      print("ДРУГОЙ РЫЧАААГ")
      break
    else:
      print(idk)
  except ValueError:
    print(idk)

Есть на просторах интернета одна гугл-таблица, в которой собрано немного про характеристики аэрогелей.
В этой прекрасной гугл-таблице насчитываются характеристики уже 82 аэрогелей
Эта программа может разбивать аэрогели на группы при помощи магии, которая зовётся MiniSOM
Магия работает по таким характеристикам, как ['Природа' 'Каж. плотность (г/см^3)' 'Уд, поверхность (м^2/г)'
 'Диаметр пор (нм)' 'Пористость (%)']
1. Кронк, жми на рычаг просмотра данных!
2. Кронк, жми на рычаг кластеризации!
3. Кронк, жми на рычаг в лабораторию!
Нажмите на рычаг: test6
Я не знаю таких цифр
1. Кронк, жми на рычаг просмотра данных!
2. Кронк, жми на рычаг кластеризации!
3. Кронк, жми на рычаг в лабораторию!
Нажмите на рычаг: 2
Результаты будут импортированы в гугл-таблицу. 
Имя листа будет: test6
Задать сигму
Одного единого универсального значения нет!



KeyboardInterrupt: Interrupted by user